In [3]:
# Code
import sys
sys.path.append('../')
from generative_formalism import *


* Loading sonnet rhythm data from `{REPO}/data/data_as_in_paper/sonnet_rhythm_data_by_sonnet_period.csv.gz`

,stanza_num,line_num,line_txt,linepart_num,parse_rank,parse_txt,is_iambic_pentameter,is_unambigously_iambic_pentameter,syll01_stress,syll02_stress,syll03_stress,syll04_stress,syll05_stress,forth_syllable_stressed,num_pos_ww,num_pos,num_sylls,perc_ww_in_meter,syll06_stress,syll07_stress,syll08_stress,syll09_stress,syll10_stress,id_hash,period_meta,subcorpus,author,author_dob,title,year,num_lines,volume,line,rhyme,genre,period,txt,group,source,data_source,prompt_type,prompt,model,temperature,model_type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
english/smithcha/Z300489005,1,1,"Mute is thy wild harp, now, O Bard sublime!",1,1,MUTE is.thy WILD harp NOW o BARD su BLIME,0,0,1,0,0,1,1.0,1,1,9,10,0.111111,1.0,0.0,1.0,0.0,1.0,2155,1750-1799 Later Eighteenth-Century,English Poetry,"Smith, Charlotte Turner, 1749-1806",1749.0,SONNET LXXXII. TO THE SHADE OF BURNS.,1779.0,14.0,Elegiac sonnets (1797–1800),"&indent;Who, amid Scotia's mountain solitude,",y,Sonnet,1700-1750,"Mute is thy wild harp, now, O Bard sublime!\n ...",C17-19,chadwyck,NaN,NaN,NaN,NaN,NaN,NaN
english/keatsjoh/Z200408021,1,1,Four seasons fill the measure of the year;,1,1,four SEA sons FILL the MEA sure OF the YEAR,1,0,1,1,0,1,0.0,1,0,10,10,0.000000,1.0,0.0,0.0,0.0,1.0,3225,1800-1834 Early Nineteenth-Century,English Poetry,"Keats, John, 1795-1821",1795.0,SONNET THE HUMAN SEASONS,1825.0,14.0,The Poetical Works (1906),&indent;There are four seasons in the mind of ...,y,Sonnet,1750-1800,Four seasons fill the measure of the year;\n ...,C17-19,chadwyck,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1cacdafb,1,2,Thou dost enthrall my heart and senses bright.,3,1,thou DOST ent HRALL my HEART and SEN ses BRIGHT,1,1,0,1,0,1,0.0,1,0,10,10,0.000000,1.0,0.0,1.0,0.0,1.0,996436,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,"Fairest maiden, with thy beauty's might,\nThou...",LLM,genai,in_paper,DO_rhyme,Write an ryhmed poem in the style of Shakespea...,ollama/llama3.1:70b,0.7,Llama
cbf384da,1,2,"Thou dost entice my heart to a sweet place,",3,1,thou DOST en TICE my HEART to.a SWEET place,0,0,0,1,0,1,0.0,1,1,9,10,0.111111,1.0,0.0,0.0,1.0,1.0,999252,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,"Fairest beauty, with thy lovely face,\nThou do...",LLM,genai,in_paper,DO_rhyme,Write an ryhmed poem in the style of Shakespea...,ollama/llama3.1:8b,0.7,Llama


In [2]:
def save_latex_table_of_period_sample():
    df_smpl = get_chadwyck_corpus_sampled_by('period')
    path = get_path_for_df(df_smpl, is_table=True)
    return get_period_subcorpus_table(df_smpl, save_latex_to=path, return_display=True)

def save_latex_table_of_rhyme_promptings():
    df = get_genai_rhyme_promptings()
    return get_rhyme_promptings_table(df, return_display=True)

def save_latex_table_of_num_poems_per_model():
    df = get_genai_rhyme_promptings()
    return get_num_poems_per_model_table(df, return_display=True)

In [3]:
# save_latex_table_of_period_sample()

In [4]:
# save_latex_table_of_num_poems_per_model()

In [5]:
# save_latex_table_of_num_poems_per_model()

In [6]:
def save_latex_table_of_num_poems_completed_per_model():
    df = get_genai_rhyme_completions(by_line=False)
    df['model2'] = df.model.apply(rename_model)
    df = df[df.model2 != '']
    df_counts = df.groupby(['model2','period']).size().unstack().T
    return get_num_poems_completed_per_model_table(df_counts, return_display=True)


In [7]:
# save_latex_table_of_num_poems_completed_per_model()


In [8]:
def save_latex_table_of_rhyming_accuracy():
    df_smpl = get_chadwyck_corpus_sampled_by('rhyme')
    df_smpl_rhyme = get_rhyme_for_sample(df_smpl, with_sample=True)
    df_preds = get_rhyming_accuracy_by_rhyme_threshold(df_smpl_rhyme)
    df_preds_tbl = df_preds.reset_index().drop('pred_by',axis=1)[['opt','precision','recall','f1_score',]].set_index('opt').sort_index()
    df_preds_tbl.columns = ['Precision', 'Recall', 'F1 score']
    df_preds_tbl.rename_axis('# Rhymes per 10 lines', inplace=True)
    df_preds_tbl = df_preds_tbl.round(2).applymap(lambda x: f'{x*100:.0f}%')
    return get_rhyme_accuracy_table(df_preds_tbl)

In [9]:
# save_latex_table_of_rhyming_accuracy()

In [10]:
def save_latex_table_of_memorization():
    df_mem = get_all_memorization_data()
    df_mem = df_mem[df_mem.found_source_corpus.str.contains('chadwyck')]
    df_mem['found'] = df_mem.found.map({False:'Not found', True:'Found'})
    df_mem['found_source'] = df_mem.found_source.map({'closed':'Closed model output', 'open':'Open training data'})
    df_mem = df_mem.rename(columns={'period':'Period'})
    df_counts = df_mem.groupby(['found_source', 'found', 'Period']).size().unstack().T
    return get_memorization_table(df_counts, return_display=True)


In [1]:
# df_sonnets = get_sonnet_rhythm_data()
# df_sonnets['source2'] = df_sonnets.source.map({'chadwyck':'Historical', 'shakespeare':'Historical', 'genai':'LLM'})

# def get_cent(period):
#     y=int(period[:2])
#     return f'C{y+1}'

# def get_model2(row):
#     if row.source2 == 'LLM':
#         return rename_model(row.model)
#     elif row.source == 'shakespeare':
#         return 'Shakespeare'
#     else:
#         return get_cent(row.period)

# df_sonnets['model2'] = df_sonnets.apply(get_model2, axis=1)
# odf = df_sonnets.groupby(['source2', 'model2']).size().reset_index()
# odf.columns = ['Source', '', '# Sonnets']
# odf = odf.set_index(['Source', ''])
# print(odf)

NameError: name 'get_sonnet_rhythm_data' is not defined

In [ ]:
# In your notebook:
def save_latex_table_of_sonnets():
    df_sonnets = get_sonnet_rhythm_data().reset_index().drop_duplicates('id').set_index('id')
    df_sonnets['source2'] = df_sonnets.source.map({'chadwyck':'Historical', 'shakespeare':'Historical', 'genai':'LLM'})
    
    def get_cent(period):
        y=int(period[:2])
        return f'C{y+1}'
    
    def get_model2(row):
        if row.source2 == 'LLM':
            return rename_model(row.model)
        elif row.source == 'shakespeare':
            return 'Shakespeare'
        else:
            return get_cent(row.period)
    
    df_sonnets['model2'] = df_sonnets.apply(get_model2, axis=1)
    odf = df_sonnets.groupby(['source2', 'model2']).size().reset_index()
    odf.columns = ['Source', '', '# Sonnets']
    odf = odf.set_index(['Source', ''])
    return get_sonnets_table(odf, return_display=True)

In [13]:
save_latex_table_of_sonnets()

* Loading sonnet rhythm data from `{REPO}/data/data_as_in_paper/sonnet_rhythm_data_by_sonnet_period.csv.gz`

* LaTeX compile failed: Command '['/Library/TeX/texbin/pdflatex', '-interaction=nonstopmode', '-halt-on-error', 'table.tex']' returned non-zero exit status 1.

'/Users/ryan/github/generative-formalism/data/data_as_in_paper/tex/table.sonnets.tex'

In [14]:
def save_latex_table_of_text_vs_instruct_completions():
    df = get_text_vs_instruct_completions()
    df_counts = df.groupby(['model','period']).size().unstack().T
    return get_text_vs_instruct_table(df_counts, return_display=True)

In [15]:
save_latex_table_of_text_vs_instruct_completions()

* Loading genai rhyme completions from `{REPO}/data/data_as_in_paper/genai_rhyme_completions.csv.gz`

Computing line similarity:   0%|          | 0/326862 [00:00<?, ?it/s]

Filtering out recognized completions:   0%|          | 0/21130 [00:00<?, ?it/s]

* Filtered out 169 recognized poems

* Converting to poem txt format (not keeping first lines from original poem)

* Loading from `/Users/ryan/github/generative-formalism/data/raw/corpus/chadwyck_corpus_metadata.csv.gz`

* LaTeX compile failed: Command '['/Library/TeX/texbin/pdflatex', '-interaction=nonstopmode', '-halt-on-error', 'table.tex']' returned non-zero exit status 1.

'/Users/ryan/github/generative-formalism/data/data_as_in_paper/tex/table.text_vs_instruct.tex'